In [1]:
import quasar

In [2]:
backend = quasar.QuasarSimulatorBackend()
nmeasurement = None

In [3]:
circuit = quasar.Circuit().Ry(0,theta=0.1).Ry(1, theta=-0.2).CZ(0,1).Ry(1, theta=+0.2).CX(1,0)
print(circuit)

T  : |0 |1|2 |3|
                
q0 : -Ry-@----X-
         |    | 
q1 : -Ry-Z-Ry-@-

T  : |0 |1|2 |3|


In [4]:
I, X, Y, Z = quasar.Pauli.IXYZ()
pauli = I[-1] + Z[0] + 2.0 * Z[1] + 3.0 * Z[0] * Z[1]
print(pauli)

+1.0*I
+1.0*Z0
+3.0*Z0*Z1
+2.0*Z1


## Derivatives

In [5]:
value = backend.run_pauli_expectation_value(
    circuit, pauli, nmeasurement=nmeasurement)
print(value)

(6.917243474764715+0j)


In [6]:
gradient = backend.run_pauli_expectation_value_gradient(
        circuit, pauli, nmeasurement=nmeasurement)
print(gradient)

[-1.64960972+0.j  0.01429936+0.j -0.01429936+0.j]


In [7]:
gradient = backend.run_pauli_expectation_value_gradient(
        circuit, pauli, parameter_indices=[1,2], nmeasurement=nmeasurement)
print(gradient)

[ 0.01429936+0.j -0.01429936+0.j]


In [8]:
hessian = backend.run_pauli_expectation_value_hessian(
        circuit, pauli, nmeasurement=nmeasurement)
print(hessian)

[-16.27556048+0.j   0.28503331+0.j  -0.28503331+0.j   0.28503331+0.j
 -11.90817496+0.j -11.85262397+0.j  -0.28503331+0.j -11.85262397+0.j
 -11.90817496+0.j]


In [9]:
hessian2 = backend.run_pauli_expectation_value_hessian(
    circuit, pauli, parameter_pair_indices=[(0,1), (1,0), (1,1)], nmeasurement=nmeasurement)
print(hessian2)

[  0.28503331+0.j   0.28503331+0.j -11.90817496+0.j]


In [10]:
pauli_expectation = quasar.run_observable_expectation_value_gradient_pauli_contraction(
    backend, circuit, pauli, param_indices=[1,2], param_coefs=[1.0, 2.0], nmeasurement=nmeasurement)
print(pauli_expectation)

AttributeError: module 'quasar' has no attribute 'run_observable_expectation_value_gradient_pauli_contraction'

## Tomography

In [ ]:
tomography = quasar.run_observable_expectation_value_tomography(
    backend, circuit, pauli, param_indices=[0,1], nmeasurement=nmeasurement)

In [ ]:
T = quasar.RotationTomography.quad_x(tomography.nparam)
print(T)

In [ ]:
import numpy as np
params = np.array(np.meshgrid(*([np.linspace(-np.pi / 2.0, +np.pi / 2.0, 50)]*2), indexing='ij'))
O = tomography.compute_observable_expectation_value(params)

In [ ]:
params_opt = tomography.optimize_jacobi_1_best()
print(params_opt[:,-1])

In [ ]:
import matplotlib.pyplot as plt
levels = np.linspace(np.min(O), np.max(O), 100)
plt.clf()
plt.contourf(params[0], params[1], O, levels, cmap=plt.get_cmap('coolwarm'))
plt.plot(T[0], T[1], 'kx')
plt.plot(params_opt[0], params_opt[1], 'm-o')
plt.xlabel(r'$\theta_0$')
plt.ylabel(r'$\theta_1$')
plt.axis('square')
plt.colorbar()
plt.show()